In [ ]:
#Dash グラフ描画
##csvファイルを読込みDBに取り込む　＊database.pyのread_data()
##DBはscrayping-appにより自動更新 *scrayping-app.py

In [9]:
#ライブラリ読込
#pandasは差分diff()算出用
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd 
import datetime

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

from assets.database import db_session
from assets.models import Data

In [19]:
#DBの各列のデータを読み込む ＊データを個別に使うわけではないので、result型でOK
data = db_session.query(Data.date, Data.start, Data.high, Data.low, Data.end, Data.adjusted).all()

dates = []
starts = []
highs = []
lows = []
ends = []
adjusteds = []

#株価がstr型になっているため、浮動小数点floatになおす
for datum in data:
    dates.append(datum.date)
    starts.append(float(datum.start.replace(",","")))
    highs.append(float(datum.high.replace(",","")))
    lows.append(float(datum.low.replace(",","")))
    ends.append(float(datum.end.replace(",","")))
    adjusteds.append(float(datum.adjusted.replace(",","")))

2020-02-12 09:52:09,110 INFO sqlalchemy.engine.base.Engine SELECT data.date AS data_date, data.start AS data_start, data.high AS data_high, data.low AS data_low, data."end" AS data_end, data.adjusted AS data_adjusted 
FROM data
2020-02-12 09:52:09,111 INFO sqlalchemy.engine.base.Engine ()


In [24]:
#差分の取得　list形式ー＞Series形式に変更
diff_starts = pd.Series(starts).diff().values
diff_highs = pd.Series(highs).diff().values
diff_lows = pd.Series(lows).diff().values
diff_ends = pd.Series(ends).diff().values
diff_adjusteds = pd.Series(adjusteds).diff().values

In [26]:
#appを作る宣言。この下にhtml要素、dcc graph要素を書いていく
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#server = app.server

app.layout = html.Div(children=[
    html.H1(children = 'グラフ描画アプリ'),
    html.Div(children =[
        dcc.Graph(
            id = 'start-price',
            figure = {
                'data':[
                    go.Scatter(
                        x=dates,
                        y=starts,
                        mode='lines+markers',
                        marker = dict(size=4),
                        opacity =0.7,
                        yaxis='y1',
                        name = '始値'
                    ),
                    go.Bar(
                        x=dates,
                        y=diff_subscribers,
                        opacity =0.7,
                        yaxis='y2',
                        name = 'diff_starts'
                    ),
                ],
                'layout':go.Layout(
                    title='stock price（始値）',
                    xaxis = dict(title='date'),
                    yaxis = dict(title='始値'', side='left',showgrid=False,
                        range = [2100, max(subscribers)+100]),
                    yaxis2 = dict(title='増減', side='right', overlaying='y', showgrid=False,
                        range = [0, max(diff_subscribers[1:])]),
                    margin=dict(l=200, r=200, b=100, t=100),    #marginの設定
                    legend = dict(
                        font = dict(),   # 凡例のフォント設定
                        x = 0.01, xanchor = 'left',  # 凡例の表示場所の設定
                        y = 0.95, yanchor = 'auto',
                        bordercolor = 'lightgray', borderwidth = 1,  # 凡例を囲む枠線の設定
                    )
                ),
            }
        ),
        dcc.Graph(
            id = 'revirews_graph',
            figure = {
                'data':[
                    go.Scatter(
                        x=dates,
                        y=reviews,
                        mode='lines+markers',
                        opacity =0.7,
                        yaxis='y1',
                        name = 'reviews'
                    ),
                    go.Bar(
                        x=dates,
                        y=diff_reviews,
                        opacity =0.7,
                        yaxis='y2',
                        name = 'diff_revirews'
                    ),
                ]
    ]),
    html.Div(children = [
        dcc.Markdown('''
        #### ------設定------
        - Heroku上でグラフを描画
        - csvローデータをDBに読込
        - WEB上から最新日のデータを取得しDBに追加（スケジューラーで自動取得登録）
        - DB ローカル：SQLite,Heroku PorstgreSQL 
        - githubでHerokuにデータをpush
        - グラフ描画：dash
        - (機能確認)下部グラフのx軸、y軸はフォント、カラー、傾きを個別に設定
        - (機能確認)凡例の位置を右上に変更。枠で囲み
        - データ取得：beautiful soup
        '''),
    ])
],

style = {
    'textAlign':'center',
    'max-width':'900',
    'margin':'0 auto'
}
)

if __name__ == '__main__':
    app.run_server(debug=True)

SyntaxError: invalid syntax (<ipython-input-26-f437de998532>, line 33)